In [53]:
import pandas as pd
import requests
import time

from datetime import datetime

In [75]:
# get station information (once for now; once daily when doing regularly?)

r = requests.get('https://gbfs.capitalbikeshare.com/gbfs/en/station_information.json')
j = r.json()
s = j['data']['stations']

stations = pd.json_normalize(s)
stations['timestamp'] = datetime.now().strftime("%H:%M:%S")

stations.to_csv('../raw_data/station_data.csv')

In [72]:
def call_dockless_api():
    
    url = 'https://gbfs.capitalbikeshare.com/gbfs/en/free_bike_status.json'
    
    r = requests.get(url)
    j = r.json()
    b = j['data']['bikes']
    
    dockless_bikes = pd.json_normalize(b)
    dockless_bikes['timestamp'] = datetime.now().strftime("%H:%M:%S")
    
    return dockless_bikes   

In [69]:
def call_docked_api():
    
    url = 'https://gbfs.capitalbikeshare.com/gbfs/en/station_status.json'
    
    r = requests.get(url)
    j = r.json()
    b = j['data']['stations']
    
    docked_bikes = pd.json_normalize(b)
    docked_bikes['timestamp'] = datetime.now().strftime("%H:%M:%S")
    
    return docked_bikes 

In [73]:
def scrape_bikes(interval):
    
    dockless_bike_data = pd.DataFrame()
    docked_bike_data = pd.DataFrame()
    timer = 0
    
    while True:
        
        current_dockless_bikes = call_dockless_api()
        current_docked_bikes = call_docked_api()
        
        dockless_bike_data = pd.concat((dockless_bike_data, current_dockless_bikes))
        docked_bike_data = pd.concat((docked_bike_data, current_docked_bikes))
        
        dockless_bike_data.to_csv('../raw_data/dockless_bike_data.csv')
        docked_bike_data.to_csv('../raw_data/docked_bike_data.csv')
        
        timer = timer + interval
        time.sleep(interval)

In [74]:
scrape_bikes(60)